## Generate grid

In [47]:
import json
import geojson as gj
import shapely as sp
import folium
import math

from uuid import uuid4

### Open file

In [12]:
with open('/code/data/geojsons/abuja.geojson', 'r') as file:
    abuja = json.load(file)

## Generate grid given zoom level of 11

In [48]:
# Maximum resolution of OSM
MAXRESOLUTION = 156543.0339

# X/Y axis offset
AXIS_OFFSET = MAXRESOLUTION * 256 / 2

def degrees_to_meters(lon, lat):
    x = lon * AXIS_OFFSET / 180
    y = math.log(math.tan((90 + lat) * math.pi / 360)) / (math.pi / 180)
    y = y * AXIS_OFFSET / 180

    return x, y

def meters_to_degrees(x,y):
    lon = x * 180 / AXIS_OFFSET
    # thanks magichim @ github for the correction
    lat = math.atan(math.exp(y * math.pi / AXIS_OFFSET)) * 360 / math.pi - 90

    return lon, lat


def create_task_feature(step, x, y):
    xmin = x * step - AXIS_OFFSET
    ymin = y * step - AXIS_OFFSET
    xmax = (x + 1) * step - AXIS_OFFSET
    ymax = (y + 1) * step - AXIS_OFFSET
    
    minlnglat = meters_to_degrees(xmin, ymin)
    maxlnglat = meters_to_degrees(xmax, ymax)

    bbox = [minlnglat[0], minlnglat[1], maxlnglat[0], maxlnglat[1]]

    return bbox


def create_grid(bbox, zoom):
    xmin, ymin, xmax, ymax = math.ceil(bbox[0]), math.ceil(bbox[1]), math.floor(bbox[2]), math.floor(bbox[3])

    step = AXIS_OFFSET / (math.pow(2, (zoom - 1)))

    xminstep = int(math.floor((xmin + AXIS_OFFSET) / step))
    xmaxstep = int(math.ceil((xmax + AXIS_OFFSET) / step))
    yminstep = int(math.floor((ymin + AXIS_OFFSET) / step))
    ymaxstep = int(math.ceil((ymax + AXIS_OFFSET) / step))    
    task_features = []

    for x in range(xminstep, xmaxstep):
        for y in range(yminstep, ymaxstep):
            # print(x, ((2**zoom-1) - y) , zoom)
            task_feature = create_task_feature(step, x, y)
            task_features.append(task_feature)
    return task_features
    

def make_grid(bbox, zoom):
    minxy = degrees_to_meters(bbox[0], bbox[1])
    maxxy = degrees_to_meters(bbox[2], bbox[3])

    bbox = [minxy[0], minxy[1], maxxy[0], maxxy[1]]
    grid = create_grid(bbox, zoom)

    return grid


def get_bounds(poly):
    polygon = Polygon(poly['geometry']['coordinates'][0])
    bounds = polygon.bounds
    
    return bounds
    
    
polygons = MultiPolygon([Polygon(p['geometry']['coordinates'][0]) for p in abuja['features']])
m = folium.Map(location=list(polygons.centroid.coords)[0], zoom_start=13)

features = []
for poly in [abuja['features'][0]]:
    bounds = get_bounds(poly)
    grid = make_grid(bounds, 17)

    # Draw polygon.
    folium.Polygon(locations=list(polygon.exterior.coords), color='red').add_to(m)

    for item in grid:
        # Get coordinates.
        poly = list(box(*item).exterior.coords)
        
        # Create geojson.
        polygon = gj.Polygon(poly, properties={"id": uuid4().hez})
        features.append(polygon)

        folium.Polygon(locations=list(poly.exterior.coords)).add_to(m)

feature_collection = gj.FeatureCollection(features)
        
        

AttributeError: 'UUID' object has no attribute 'hez'

In [36]:
test = abuja['features'][0]
boom = gj.Polygon(geometry=test["geometry"]["coordinates"], properties={"name": "AAAAA"})

alala = list(poly.exterior.coords)
gj.Polygon(alala, properites={'name': 'lalalla'})

{"coordinates": [[7.319641, 9.145486], [7.319641, 9.148198], [7.316895, 9.148198], [7.316895, 9.145486], [7.319641, 9.145486]], "properites": {"name": "lalalla"}, "type": "Polygon"}

In [46]:
import uuid
test = uuid.uuid4().hex
test

'6d73ca0a6c184afc82954eb02c06a7c7'